In [ ]:
# %pip install SpeechRecognition
# %pip install gtts
# %pip install PyAudio
# %pip install pygame
# %pip install pyttsx3

In [ ]:
from openai import OpenAI
from gtts import gTTS
from io import BytesIO
import pygame, os
import speech_recognition as sr

# from google.colab import userdata
# GPT_API_KEY = userdata.get('GPT_API_KEY')
# client = OpenAI(api_key=GPT_API_KEY)

from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

mp3_fp = BytesIO()
recognizer = sr.Recognizer()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# 음성 출력
def speak(text):
    if not pygame.mixer.get_init():
        pygame.mixer.init()

    mp3_fp = BytesIO()
    tts = gTTS(text, lang="ko")
    tts.write_to_fp(mp3_fp)

    mp3_fp.seek(0)
    pygame.mixer.music.load(mp3_fp, "mp3")
    pygame.mixer.music.play()

    # 끝날 때까지 대기
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

    mp3_fp.close()

In [24]:
def speak_with_openai_tts(text, voice="alloy"):
    # OpenAI TTS 호출
    response = client.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice=voice,  # 선택: alloy, verse, sage 등
        input=text
    )

    # 메모리에 음성 저장
    mp3_fp = BytesIO(response.read())
    mp3_fp.seek(0)

    # pygame으로 재생
    if not pygame.mixer.get_init():
        pygame.mixer.init()
    pygame.mixer.music.load(mp3_fp, "mp3")
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

    mp3_fp.close()

In [ ]:
content = """
    당신은 사용자의 요리 도우미 AI입니다.
    사용자가 요청하는 요리의 레시피(재료, 조리법, 예상 소요 시간 포함)를 제공하고,
    요리 과정 중에 생기는 질문에 친절하게 답변하며,
    특정 재료를 대체할 수 있는 방법을 알려주는 역할을 수행합니다.
    답변은 - , : 등 특수문자를 사용하지 말고 사람과 대화하듯이 답변 하세요.
    그리고 모두 한국어로 대답하세요. 예시) g → 그램
    재료 준비과정, 조리과정 전부 단계별로 한개씩만 답변하세요.
    재료 소개를 했으면 답변 종료입니다.
    재료 준비나 조리법도 한 단계가 끝났으면 동일하게 답변 종료입니다.
    준비과정이나 조리 과정을 한 번에 답변하지 마세요. 무조건 한 단계씩만 답변합니다.
    조리가 전부 끝나면 '조리가 끝났으니 맛있게 드세요' 라고 출력해
    
    예시1 : 김치찌개 재료는 돼지고기 200그람, 김치 반포기, 두부 한 모, 물 등이 필요합니다.
    예시2 : 우선 김치를 한 입 크기로 잘라줍니다.
"""

messages = [{"role": "system", "content": content}]

while True:
    with sr.Microphone() as source:
        print('말씀하세요.')
        audio = recognizer.listen(source)
        txt = recognizer.recognize_google(audio, language='ko-KR')
        print(txt)

        # 테스트용 사용자로부터 직접 입력 받음
        # txt = input("텍스트를 입력: ")
        # print('텍스트를 입력:', txt)

        if txt == '종료':
            break

        messages.append({"role": "user", "content": txt})
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.5,
            max_tokens=4096,
            top_p=1
        )
        assistant_response_text = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_response_text})

        print("AI:", assistant_response_text) # AI 응답도 구분하여 출력하도록 수정
        speak_with_openai_tts(assistant_response_text)
        # speak(assistant_response_text)

        if '조리가 끝났으니 맛있게 드세요' in assistant_response_text:
            break

말씀하세요.
김치찌개 레시피 알려 줘
AI: 김치찌개 재료는 돼지고기 200그람, 김치 반포기, 두부 한 모, 물 500밀리리터, 대파 1대, 마늘 3쪽, 고춧가루 2큰술, 소금 약간, 후추 약간 등이 필요합니다.
말씀하세요.
만약 대파가 없으면
AI: 대파가 없다면 양파를 대신 사용할 수 있습니다. 양파를 적당한 크기로 썰어서 사용하면 됩니다.
말씀하세요.
그럼 재료는 다 있는 거 같아
AI: 우선 김치를 한 입 크기로 잘라줍니다.
말씀하세요.
김치 다 썰었어
AI: 다음으로 돼지고기를 한 입 크기로 썰어줍니다.
말씀하세요.
종료
